In [ ]:
!pip install scikit-surprise

In [ ]:
import pandas as pd
import surprise
import numpy as np

In [ ]:
# 이번엔 KNN돌려볼건데 계속 램이 터져서... 일단 랜덤데이터로만 고
final_data_random = pd.read_csv('/content/drive/MyDrive/212343_final_data_random.csv', index_col=0)

In [ ]:
#... 근데 해보니까 1000000만개 학습은 어림도 없고, 1000000개도 램 터진다. 데이터 개수를 겁나게 줄이겠다.
import random
KNN_random_index = random.sample(list(final_data_random.index), 50000)
KNN_random = final_data_random.loc[KNN_random_index]
KNN_random

,userId,movieId,rating,timestamp
615301,230658,48516,3.5,1309650129
120921,48656,4034,3.5,1150221231
33988,50840,110,3.0,1209248122
679069,248563,1988,2.0,901622632
437076,19421,2511,3.0,965172593
...,...,...,...,...
628364,122834,103339,4.0,1430231439
680082,8772,2455,3.5,1105401340
67194,281340,1233,3.0,1002580558
30732,261508,1204,5.0,1107585229


In [ ]:
# 영화 데이터도 고
movie_df = pd.read_csv('/content/drive/MyDrive/movies.csv')

In [ ]:
# 나눠봅시다잉
from surprise import SVD, Dataset, accuracy
from surprise.model_selection import train_test_split
from surprise.dataset import DatasetAutoFolds
from surprise import Reader

col = 'user item rating'
reader = Reader(line_format=col, sep='\t') # 반드시 사용자-아이템-평점 순서로
data = Dataset.load_from_df(KNN_random[['userId', 'movieId', 'rating']], reader=reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

In [ ]:
# KNNBasic과 KNNWithMeans의 차이점은 k명의 유사 사용자들이 item에 대해 평가한 평점들을 그대로 가중합 하지 않는다는 것. 
# 우리가 예측하려는 사용자의 평균을 더해주는 방법으로 사용자의 특성을 좀 더 부가하는 방법이라고 이해하면 편하다.
# 출처 : https://danthetech.netlify.app/DataScience/how-does-recommendation-algorithms-work-using-surpriselib#knnwithmeans
from surprise import KNNWithMeans 
sim_options = {
    'name': 'cosine', # ['msd', 'pearson_baseline', 'pearson']
    'user_based': True, # user를 base로 할것인지 item을 base로 할 것인지 결정. 보통은 아이템 기반의 추천 방식을 사용한다고 한다.
    # 왜냐하면 사용자 기반의 추천방식은 특정 사용자를 비슷한 다른 사용자들과 특성이 거의 비슷함을 가정하는데, 이는 개개인마다 기호, 성격이 각기 너무 다른 현실세계의 변수를 제대로 반영하지 못하기 때문이라고.
    # 그래도 여기서는 사용자 기반으로 해보겠음. KNNWithMeans와 연계하여
    'min_support': 150, # u, v간 교집합 원소 최소 갯수를 설정한다. 이 경우는 150개 이상의 공통된 아이템(or 사용자)가 존재하면 유사도를 계산하지만, 그 미만의 경우에는 sim(u, v) == 0로 설정한다.
    # 왜 150개로 했냐고? 랜덤데이터 1000000개 / 212343 데이터 7000개라 그냥 나눈 값 근사치로
    'shrinkage': 0, # 이 파라미터는 pearson_baseline을 사용할 경우에만 해당된다.
}
algo = KNNWithMeans(sim_options=sim_options)
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
# 10000000개에서 50000개까지 줄여야 학습이 되는구나... 그것도 그나마 트레인셋 75%면 37000개 정도네... 결과나 보자.
test_pred = algo.test(testset)
test_pred[:2]

[Prediction(uid=281785, iid=608, r_ui=3.0, est=3.5693733333333335, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=79570, iid=43936, r_ui=3.0, est=3.5693733333333335, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})]

In [ ]:
# 데이터가 많이 없다보니까 오차가 높구먼... min_support를 줄여봐야겠다.
surprise.accuracy.rmse(test_pred, verbose=False)

1.1178696680079854

In [ ]:
sim_options = {
    'name': 'pearson', # ['msd', 'pearson_baseline', 'cosine'] # 이번엔 pearson으로
    'user_based': True, 
    'min_support': 5, # 5로 해보겠다.
    'shrinkage': 0, 
}
algo = KNNWithMeans(sim_options=sim_options)
algo.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred = algo.test(testset)
test_pred[:2]

[Prediction(uid=281785, iid=608, r_ui=3.0, est=3.5693733333333335, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=79570, iid=43936, r_ui=3.0, est=3.5693733333333335, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})]

In [ ]:
# ...? 똑같네...? 그러면 이번엔 item_base, msd로 해보자.
surprise.accuracy.rmse(test_pred, verbose=False)

1.1178696680079854

In [ ]:
sim_options = {
    'name': 'msd', # ['pearson', 'pearson_baseline', 'cosine']
    'item_based': True, 
    'min_support': 5,
    'shrinkage': 0, 
}
algo = KNNWithMeans(sim_options=sim_options)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred = algo.test(testset)
test_pred[:2]

[Prediction(uid=281785, iid=608, r_ui=3.0, est=3.5693733333333335, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=79570, iid=43936, r_ui=3.0, est=3.5693733333333335, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})]

In [ ]:
surprise.accuracy.rmse(test_pred, verbose=False)

1.1178696680079854

In [ ]:
# 그냥 똑같구먼.... 그럼 이번엔 KNNBasic, pearson_baseline, item_based = True로 해보자.
from surprise import KNNBasic
sim_options = {
    'name': 'pearson_baseline', # ['pearson', 'msd', 'cosine']
    'item_based': True, 
    'min_support': 5,
    'shrinkage': 100, # pearson_baseline은 얘를 줘야함.
    # 피어슨-베이스라인 유사도는 벡터의 차원 즉, 두 사용자나 상품에 공통적으로 있는 평점 원소의 갯수를 이용하여 정규화를 하는 shrinkage를 추가하여 사용한다.
    # 출처 : https://datascienceschool.net/03%20machine%20learning/07.01%20%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C.html
}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
# 램이 진짜 터지려고한다....
test_pred = algo.test(testset)
test_pred[:2]

[Prediction(uid=281785, iid=608, r_ui=3.0, est=3.5693733333333335, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=79570, iid=43936, r_ui=3.0, est=3.5693733333333335, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})]

In [ ]:
# 조금 줄긴 했네... 근데 이거로는 212343에게 추천은 불가능 할것 같다. 랜덤으로 뽑은 데이터에 212343이 없을수도 있기 때문.
# 그냥 재미로 했다고 생각하자.
surprise.accuracy.rmse(test_pred, verbose=False)

1.0505237313095046